In [3]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
# load ANN trained model, scaler pickle file, one hot encoder and label encoder pickle file
model = load_model('model.h5')

# load encoders and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_country.pkl','rb') as file:
    onehot_encoder_country=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)



In [6]:
# Example input data
input_data = {
    'credit_score': 600,
    'country': 'France',
    'gender': 'Male',
    'age': 40,
    'tenure':3,
    'balance': 60000,
    'products_number': 2,
    'credit_card': 1,
    'active_member': 1,
    'estimated_salary': 50000
}


In [9]:
input_df = pd.DataFrame([input_data])
input_df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
# encode gender here
input_df['gender']=label_encoder_gender.transform(input_df['gender'])
input_df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
# encode geography
encoded_country = onehot_encoder_country.transform(input_df[['country']])
encoded_df = pd.DataFrame(encoded_country, columns=onehot_encoder_country.get_feature_names_out(['country']))
input_df = pd.concat([input_df.drop(columns=['country'], axis=1), encoded_df], axis=1)
input_df

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
# scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [14]:
# make predictions
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 99ms/step


array([[0.0234987]], dtype=float32)

In [15]:
pred_prob = prediction[0][0]
pred_prob

0.0234987

In [16]:
if pred_prob > 0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
